In [1]:
import os
import pandas as pd
import numpy as np
from geopy.distance import geodesic

In [2]:
train_path = '../data/hy_round2_train_20200225'

In [3]:
train_df_list = []
for file_name in os.listdir(train_path):
    df = pd.read_csv(os.path.join(train_path, file_name))
    train_df_list.append(df)

train_df = pd.concat(train_df_list)

train_df['time'] = pd.to_datetime(train_df['time'], format='%m%d %H:%M:%S')

all_df = pd.concat([train_df], sort=False)

/Users/yitinglin/Projects/fishing-classification/env/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [5]:
shape_list = []
for ship_id, group in all_df.groupby('渔船ID'):
    shape_list.append(group.shape[0])

In [8]:
np.mean(shape_list)

598.730557256583

In [28]:
from tslearn.barycenters import dtw_barycenter_averaging, dtw_barycenter_averaging_subgradient

for ship_id, group in all_df.groupby('渔船ID'):
    lat = group['lat'].values
    lon = group['lon'].values
    v = group['速度'].values
    d = group['方向'].values
    center1 = dtw_barycenter_averaging([lat, lon, v, d], max_iter=10, barycenter_size=4)
    center2 = dtw_barycenter_averaging_subgradient([lat, lon, v, d], max_iter=10, barycenter_size=4)
#     lon_center = dtw_barycenter_averaging(lon, barycenter_size=1)
#     v_center = dtw_barycenter_averaging(v, barycenter_size=1)
#     d_center = dtw_barycenter_averaging(d, barycenter_size=1)

    print(np.mean(center1.flatten()), np.mean(center2.flatten()))


57.467282401122276 -4790.178583670846
45.65091239426052 -527.7984368288396
51.19875259802629 -77.51443858176177
88.5515147317504 -3757.39627697595
62.76639439809574 -4683.245313885639
60.73917050239343 -2210.2103335030797
41.516086534289805 -3658.235727142012
66.71315549537302 -732.9133415131869
83.45421579919281 -0.5790777943311127
97.99217005246075 -607.6474640300385
58.04223824071853 3146.372930863439
74.06783246095573 312.9327789991737
113.70385909731169 441.41584796738096
45.22933794327933 -2652.5276575735043
69.91562080700643 -2485.5405511905774
87.42763303056145 -2090.367679226132
99.27997883379811 2002.4870455611726
106.80680199421691 514.2317580498684
63.30352879617118 -5878.973758809185
44.0967205882003 2552.618267209129
71.01066835369483 5512.7091254282
74.87585565435016 -1354.4477276832124
77.97559182941613 -2350.2871235687157
82.4359338411357 233.3739540605907
67.49532384944297 475.9585832997955
41.179282194975926 -2999.933827580174
67.16113251479698 -6122.011487136563
95.

95.60048082080513 2278.922458895423
64.00555823680176 -834.0356421824736
101.1943178434864 -1323.7852442732649
91.97461350327971 -25781.768256539384
68.34931835048177 -9388.816427808417
62.98978789572192 -12387.099337050653
57.54778648202676 6492.9526490391845
53.16489020182834 -1440.8687187177184
83.59912769897375 1661.0146446617048
60.675267543071065 -1055.7614597233128
82.92958613909107 1982.175497636784
91.70805223171206 41.799625117231614
74.18814459734571 1124.9927490563525
146.77918059525527 -212.58245764738757
54.54434339194681 -425.55088804777114
69.2197424197073 -219.82329298283832
52.547566520930296 3237.5753245134756
76.10197523322122 1894.7431039817557
92.70768740871364 -1210.7516396922852
80.72727843698856 -6377.73635993266
105.16629477335215 -2647.4471076279665
55.58989038212184 671.9298443154142
94.63627418093438 -914.448885784745
63.900950333025996 78.50103096320117
43.99756076986152 1111.2794704127095
86.62182001093922 211.06871922854214
104.96244727859579 -167.741234

KeyboardInterrupt: 

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('./feature/train.csv', index_col='id')

In [3]:
train_df.shape

(8166, 1568)

In [4]:
import lightgbm as lgb

In [6]:
X_train = train_df.drop(columns=['type'])
y_train = train_df['type']

base_model = lgb.LGBMClassifier(n_estimators=500, subsample=0.8)
base_model.fit(X_train.values, y_train)

selected_columns = X_train.columns[np.argsort(base_model.feature_importances_)[::-1][:50]]

In [7]:
selected_columns = X_train.columns[np.argsort(base_model.feature_importances_)[::-1][:500]]
selected_columns

Index(['lat__quantile__q_0.9', 'lon__quantile__q_0.1',
       '速度__number_crossing_m__m_1', 'lon__minimum', 'lat__maximum',
       'lat__quantile__q_0.8', 'lon__quantile__q_0.2', 'lat__minimum',
       'lon__quantile__q_0.4', 'lon__maximum',
       ...
       '方向__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_5',
       '速度__fft_coefficient__coeff_6__attr_"abs"',
       '方向__fft_coefficient__coeff_59__attr_"abs"', '方向__count_above_mean',
       '方向__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.0',
       '速度__fft_coefficient__coeff_19__attr_"abs"',
       '速度__fft_coefficient__coeff_73__attr_"abs"',
       'lat__fft_coefficient__coeff_73__attr_"angle"',
       'lon__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.8',
       'lon__autocorrelation__lag_2'],
      dtype='object', length=500)

In [8]:
X_train_filtered = X_train[selected_columns]

In [9]:
X_train_filtered.shape

(8166, 500)

In [10]:
import tsfresh
kind_to_fc_parameters = tsfresh.feature_extraction.settings.from_columns(X_train_filtered)

In [11]:
kind_to_fc_parameters

{'lat': {'quantile': [{'q': 0.9},
   {'q': 0.8},
   {'q': 0.3},
   {'q': 0.6},
   {'q': 0.2},
   {'q': 0.7},
   {'q': 0.1},
   {'q': 0.4}],
  'maximum': None,
  'minimum': None,
  'number_cwt_peaks': [{'n': 1}],
  'fft_coefficient': [{'coeff': 5, 'attr': 'real'},
   {'coeff': 74, 'attr': 'angle'},
   {'coeff': 5, 'attr': 'angle'},
   {'coeff': 66, 'attr': 'angle'},
   {'coeff': 93, 'attr': 'angle'},
   {'coeff': 10, 'attr': 'imag'},
   {'coeff': 15, 'attr': 'real'},
   {'coeff': 10, 'attr': 'real'},
   {'coeff': 94, 'attr': 'angle'},
   {'coeff': 86, 'attr': 'angle'},
   {'coeff': 96, 'attr': 'angle'},
   {'coeff': 9, 'attr': 'angle'},
   {'coeff': 91, 'attr': 'angle'},
   {'coeff': 83, 'attr': 'angle'},
   {'coeff': 88, 'attr': 'angle'},
   {'coeff': 99, 'attr': 'angle'},
   {'coeff': 73, 'attr': 'angle'}],
  'median': None,
  'abs_energy': None,
  'cwt_coefficients': [{'widths': (2, 5, 10, 20), 'coeff': 7, 'w': 2},
   {'widths': (2, 5, 10, 20), 'coeff': 14, 'w': 5},
   {'widths': (2,